In [1]:
import ray

if not ray.is_initialized():
    ray.shutdown()
    ray.init(dashboard_host="0.0.0.0")

ray.available_resources()

2025-03-11 13:12:40,323	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://172.23.138.198:8265 


{'accelerator_type:G': 1.0,
 'node:__internal_head__': 1.0,
 'CPU': 8.0,
 'object_store_memory': 2871375052.0,
 'memory': 5742750107.0,
 'node:172.23.138.198': 1.0,
 'GPU': 1.0}

In [10]:
import pandas as pd
import numpy as np

xl = 0
xu = 1
step = 10 ** -3
x = np.arange(xl, xu + step, step)
x[0:10]

array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009])

In [20]:
from pymoo.termination import get_termination
from pymoo.optimize import minimize
import logging
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.core.variable import Choice, Real
from pymoo.algorithms.moo.nsga2 import RankAndCrowding
from pymoo.core.mixed import MixedVariableGA, MixedVariableMating, MixedVariableDuplicateElimination, \
    MixedVariableSampling
import numpy as np
import pandas as pd
from pymoo.core.problem import ElementwiseProblem, RayParallelization
from pymoo.algorithms.moo.age2 import AGEMOEA2

from pymoo.problems.many import C1DTLZ1, DC1DTLZ1, DC1DTLZ3, DC2DTLZ1, DC2DTLZ3, DC3DTLZ1, DC3DTLZ3, C1DTLZ3, \
    C2DTLZ2, C3DTLZ1, C3DTLZ4, DTLZ1
from pymoo.indicators.igd_plus import IGDPlus

origin_problems = [C1DTLZ1, DC1DTLZ1, DC1DTLZ3, DC2DTLZ1, DC2DTLZ3, DC3DTLZ1, DC3DTLZ3, C1DTLZ3, C2DTLZ2, C3DTLZ1,
                   C3DTLZ4, DTLZ1]

runner = RayParallelization(job_resources={})
problems = {}

for origin_problem in origin_problems:
    template = origin_problem()

    custom_origin_problem = type(origin_problem.__name__ + "OriginProblem", (origin_problem,), {})
    problems[custom_origin_problem.__name__] = custom_origin_problem(elementwise_runner=runner)

    def _evaluate(self, X, out, *args, **kwargs):
        ported_X = np.zeros((X.shape[0], len(X[0])))
        for i, x in enumerate(range(ported_X.shape[0])):
            for key, value in X[i].items():
               ported_X[i, key] = value

        super(self.__class__, self)._evaluate(ported_X, out, *args, **kwargs)

    def custom_real_problem_constructor(self, *args, **kwargs):
        kwargs["vars"] = {i: Real(bounds=(template.xl[i], template.xu[i])) for i in range(0, template.n_var)}
        super(self.__class__, self).__init__(**kwargs)


    custom_real_problem = type(origin_problem.__name__ + "RealMixedProblem", (origin_problem,), {
        "__init__": custom_real_problem_constructor,
        "_evaluate": _evaluate,
    })
    problems[custom_real_problem.__name__] = custom_real_problem(elementwise_runner=runner)


    def custom_choice_problem_constructor(self, *args, **kwargs):
        step = 10 ** -6
        kwargs["vars"] = {i: Choice(options=np.arange(template.xl[i], template.xu[i] + step, step)) for i in
                          range(0, template.n_var)}
        super(self.__class__, self).__init__(**kwargs)


    custom_choice_problem = type(origin_problem.__name__ + "ChoiceMixedProblem", (origin_problem,), {
        "__init__": custom_choice_problem_constructor,
        "_evaluate": _evaluate,
    })
    problems[custom_choice_problem.__name__] = custom_choice_problem(elementwise_runner=runner)


    def custom_combine_problem_constructor(self, *args, **kwargs):
        if template.n_var < 2:
            raise ValueError("The number of variables must be greater than or equal to 2.")

        i_mid = int(template.n_var / 2)
        step = 10 ** -6
        kwargs["vars"] = {
            **{i: Real(bounds=(template.xl[i], template.xu[i])) for i in range(0, i_mid)},
            **{i: Choice(options=np.arange(template.xl[i], template.xu[i] + step, step))
               for i in range(i_mid, template.n_var)}
        }
        super(self.__class__, self).__init__(**kwargs)


    custom_combine_problem = type(origin_problem.__name__ + "CombineMixedProblem", (origin_problem,), {
        "__init__": custom_combine_problem_constructor,
        "_evaluate": _evaluate,
    })
    problems[custom_combine_problem.__name__] = custom_combine_problem(elementwise_runner=runner)


mixed_smsemoa_algorithm = SMSEMOA(
    sampling=MixedVariableSampling(),
    mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
    eliminate_duplicates=MixedVariableDuplicateElimination(),
)

mixed_ga_algorithm = MixedVariableGA(
    survival=RankAndCrowding(),
)

mixed_agemoea2_algorithm = AGEMOEA2(
    sampling=MixedVariableSampling(),
    mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
    eliminate_duplicates=MixedVariableDuplicateElimination(),
)

agemoea2_algorithm = AGEMOEA2()

algorithms = {
    "origin_agemoea2_algorithm": agemoea2_algorithm,
    "mixed_smsemoa_algorithm": mixed_smsemoa_algorithm,
    "mixed_ga_algorithm": mixed_ga_algorithm,
    "mixed_agemoea2_algorithm": mixed_agemoea2_algorithm
}


def run(input):
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.info(f"input: {input['label']}")
    termination = get_termination("time", "00:10:10")
    result = minimize(
        problem=input["problem"],
        algorithm=input["algorithm"],
        verbose=True,
        seed=1
    )
    output = {
        **input,
        "result": result,
    }
    return output



In [21]:
inputs = [
    {"label": {"problem": problem_key, "algorithm": algorithm_key}, "problem": problem_value,
     "algorithm": algorithm_value}
    for problem_key, problem_value in problems.items()
    for algorithm_key, algorithm_value in algorithms.items()
    if ("mixed" not in algorithm_key and "OriginProblem" in problem_key)
    or ("mixed" in algorithm_key and "OriginProblem" not in problem_key)
]
inputs

[{'label': {'problem': 'C1DTLZ1OriginProblem',
   'algorithm': 'origin_agemoea2_algorithm'},
  'problem': <__main__.C1DTLZ1OriginProblem at 0x7f5d10170e60>,
  'algorithm': <pymoo.algorithms.moo.age2.AGEMOEA2 at 0x7f5c47f29f70>},
 {'label': {'problem': 'C1DTLZ1RealMixedProblem',
   'algorithm': 'mixed_smsemoa_algorithm'},
  'problem': <__main__.C1DTLZ1RealMixedProblem at 0x7f5c48056b70>,
  'algorithm': <pymoo.algorithms.moo.sms.SMSEMOA at 0x7f5c47f290a0>},
 {'label': {'problem': 'C1DTLZ1RealMixedProblem',
   'algorithm': 'mixed_ga_algorithm'},
  'problem': <__main__.C1DTLZ1RealMixedProblem at 0x7f5c48056b70>,
  'algorithm': <pymoo.core.mixed.MixedVariableGA at 0x7f5c47f292b0>},
 {'label': {'problem': 'C1DTLZ1RealMixedProblem',
   'algorithm': 'mixed_agemoea2_algorithm'},
  'problem': <__main__.C1DTLZ1RealMixedProblem at 0x7f5c48056b70>,
  'algorithm': <pymoo.algorithms.moo.age2.AGEMOEA2 at 0x7f5c47f29ee0>},
 {'label': {'problem': 'C1DTLZ1ChoiceMixedProblem',
   'algorithm': 'mixed_smsem

In [ ]:
tasks = [ray.remote(run).remote(input) for input in inputs]
outputs = ray.get(tasks)

[2025-03-11 13:54:21,076 E 780569 780945] core_worker.cc:936: :info_message: Attempting to recover 61 lost objects by resubmitting their tasks. To disable object reconstruction, set @ray.remote(max_retries=0).


In [9]:
import copy

copied_outputs = copy.deepcopy(outputs)

In [12]:
df_records = []

for output in copied_outputs:
    result = output["result"]
    label = output["label"]

    true_pf = output["problem"].pareto_front()
    predicted_pf = result.F

    if predicted_pf is None:
        df_records.append({
            "problem": label["problem"],
            "algorithm": label["algorithm"],
            "IGD+": None,
            "IGD+ Worst": None,
            "IGD+ Relative to Worst": None
        })
        continue

    igdp = IGDPlus(true_pf)
    score = igdp(predicted_pf)

    worst_pf = np.max(true_pf, axis=0)
    igdp_worst = IGDPlus(true_pf)
    score_worst = igdp(worst_pf)

    relative_score = score / score_worst

    df_records.append({
        "problem": label["problem"],
        "algorithm": label["algorithm"],
        "IGD+": score,
        "IGD+ Worst": score_worst,
        "IGD+ Relative to Worst": relative_score
    })

df = pd.DataFrame.from_records(df_records)
df


,problem,algorithm,IGD+,IGD+ Worst,IGD+ Relative to Worst
0,C1DTLZ1RealMixedProblem,mixed_agemoea2_algorithm,0.013895,0.618395,0.022470
1,C1DTLZ1ChoiceMixedProblem,mixed_agemoea2_algorithm,NaN,NaN,NaN
2,C1DTLZ1CombineMixedProblem,mixed_agemoea2_algorithm,NaN,NaN,NaN
3,DC1DTLZ1RealMixedProblem,mixed_agemoea2_algorithm,0.042347,0.618395,0.068479
4,DC1DTLZ1ChoiceMixedProblem,mixed_agemoea2_algorithm,17.205546,0.618395,27.822912
5,DC1DTLZ1CombineMixedProblem,mixed_agemoea2_algorithm,6.727411,0.618395,10.878827
6,DC1DTLZ3RealMixedProblem,mixed_agemoea2_algorithm,0.082615,1.075293,0.076830
7,DC1DTLZ3ChoiceMixedProblem,mixed_agemoea2_algorithm,63.027523,1.075293,58.614254
8,DC1DTLZ3CombineMixedProblem,mixed_agemoea2_algorithm,17.799457,1.075293,16.553116
9,DC2DTLZ1RealMixedProblem,mixed_agemoea2_algorithm,NaN,NaN,NaN


In [13]:
df.describe()

,IGD+,IGD+ Worst,IGD+ Relative to Worst
count,28.000000,28.000000,28.000000
mean,15.458537,1.010491,16.858527
std,28.394129,0.540566,27.514596
min,0.013165,0.618395,0.014424
25%,0.057005,0.618395,0.043696
50%,0.842326,1.075293,1.145132
75%,18.041201,1.075293,28.453935
max,129.078050,2.402783,120.039837
